# Basic ML: Phonocardiograms

Author: Jake Dumbauld <br>
Contact: jacobmilodumbauld@gmail.com<br>
Date: 3.15.22

## Imports

In [2]:
#imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import time
import re

## Framework

How to Approach the Problem:
- Current plan:
    - Intend to initially fit an RNN to the raw, unpadded signal data and see how it fares. 
    - Also fit a CNN to the MFCC data and see how that fares
    - Stack the outputs of those two with a third CNN that incorporates the patient information provided in the annotations.
    
- Questions:
    - Does this plan make sense?
    - How should I go about optimizing my parameters & deciding on network architecture?
    - 

In [ ]:
# use glob to extract files. 

- do LTSM
- 1D CNN 


- Trim sampling rate to 25% of what it is now.

RNN approach, concatenate patient information into a 1D vector to feed into the model

Look into an ensemble/aggregation model. 

Can incorporate patient info into CNN as well, just need to pad the vector

Look into training accelerometer data on LTSMs. 

Look into varying model hyperparameters based on frequency of the data.

Look into example architectures for CNN that are used on MFCCs

make sure to train a dumb feed forward model

Try 1024 for an input layer. 

1. Train a simple feed forward on the padded and transformed data. 
- MFCC data with patient demo info into feed forward. 
2. RNN, with patient info
- First train without, then train with. 
3. CNN on MFCCs with patient info. 
- First train out, then train with. 